In [70]:
import pandas as pd

train = pd.read_csv("../data/movie/labeledTrainData.tsv",
                    header = 0,
                    delimiter="\t",
                    quoting=3)
test = pd.read_csv("../data/movie/testData.tsv",
                    header = 0,
                    delimiter="\t",
                    quoting=3)
unlabeld_train = pd.read_csv("../data/movie/unlabeledTrainData.tsv",
                    header = 0,
                    delimiter="\t",
                    quoting=3)

In [71]:
display(test.head(1))
display(train.head(1))
display(unlabeld_train.head(1))

,id,review
0,"""12311_10""","""Naturally in a film who's main themes are of ..."


,id,sentiment,review
0,"""5814_8""",1,"""With all this stuff going down at the moment ..."


,id,review
0,"""9999_0""","""Watching Time Chasers, it obvious that it was..."


In [105]:
from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords

def review_to_wordlist(review, remove_stopwords=False):
    # 1. HTML 태그 지우기
    review_text = BeautifulSoup(review).get_text()
    # 2. 알파벳 빼고 다 지움
    review_text = re.sub("[^a-zA-Z]"," ", review_text)
    # 3. 소문자로 바꾸고 단어 분리
    words = review_text.lower().split()
    # 4. 정지단어 제거
    if remove_stopwords:
        stops = set(stopwordswords.word("english"))
        words = [w for w in words if not w in stops]
    
    return (words)

In [101]:
rr = tokenizer.tokenize("I am a boy   ".strip())
print(rr)
"".join(rr)

['I am a boy']


'I am a boy'

In [108]:
import nltk.data

tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

def review_to_sentences(review, tokenizer, remove_stopwords=False):
    raw_sentences = tokenizer.tokenize(review.strip())
    sentences = []
    for raw_sentence in raw_sentences:
        if len(raw_sentence) >0:
            sentences.append(review_to_wordlist(raw_sentence, remove_stopwords))
    return sentences

In [112]:
sentences = []

for review in train["review"]:
    sentences += review_to_sentences(review, tokenizer)
    
for reveiw in unlabeld_train["review"]:
    sentences += review_to_sentences(review, tokenizer)

C:\ProgramData\Anaconda3\envs\gpu_env\lib\site-packages\bs4\__init__.py:272: UserWarning: "b'.'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)
C:\ProgramData\Anaconda3\envs\gpu_env\lib\site-packages\bs4\__init__.py:272: UserWarning: "b'...'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)
C:\ProgramData\Anaconda3\envs\gpu_env\lib\site-packages\bs4\__init__.py:335: UserWarning: "http://www.happierabroad.com"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup


In [113]:
print(len(sentences))

716551


In [114]:
print(sentences[0])

['with', 'all', 'this', 'stuff', 'going', 'down', 'at', 'the', 'moment', 'with', 'mj', 'i', 've', 'started', 'listening', 'to', 'his', 'music', 'watching', 'the', 'odd', 'documentary', 'here', 'and', 'there', 'watched', 'the', 'wiz', 'and', 'watched', 'moonwalker', 'again']


In [ ]:
# 아키텍처 : 아키텍처 옵션은 건너 뛰기 (기본값) 또는 연속적인 단어 모음입니다. 우리는 건너 뛰기 그램이 매우 느리지 만 더 나은 결과를 산출한다는 것을 발견했습니다.
# 교육 알고리즘 : 계층 적 softmax (기본값) 또는 음의 샘플링. 우리에게는 기본값이 잘 작동했습니다.
# 빈번한 단어의 다운 샘플링 : Google 문서는 0.00001에서 0.001 사이의 값을 권장합니다. 우리에게는 0.001에 가까운 값이 최종 모델의 정확성을 향상시키는 것으로 보였습니다.
# 단어 벡터 차원 : 더 많은 기능을 사용하면 런타임이 길어지고 항상 좋은 것은 아니지만 종종 더 나은 모델이됩니다. 합리적인 값은 수십에서 수백 개가 될 수 있습니다. 300을 사용했습니다.
# 컨텍스트 / 창 크기 : 트레이닝 알고리즘이 고려해야하는 컨텍스트의 단어 수는 얼마입니까? 10은 계층 적 소프트 맥스 (더 나은 점은 포인트가 더 많음)에서 잘 작동하는 것처럼 보입니다.
# Worker threads : 실행할 병렬 프로세스의 수. 이것은 컴퓨터마다 다르지만 대부분의 시스템에서 4에서 6 사이의 값을 사용해야합니다.
# 최소 단어 수 : 어휘의 크기를 의미있는 단어로 제한하는 데 도움이됩니다. 적어도이 모든 경우에 여러 번 발생하지 않는 단어는 무시됩니다. 합리적인 값은 10에서 100 사이가 될 수 있습니다.이 경우 각 영화가 30 번 발생하기 때문에 개별 영화 제목에 너무 많은 중요성을 부여하지 않기 위해 최소 단어 수를 40으로 설정합니다. 그 결과 전체 어휘 크기는 약 15,000 단어가되었습니다. 값이 높을수록 실행 시간이 제한됩니다.

In [116]:
%%time

num_features = 300 # word vector 차원
min_word_count = 40 # 최소 단어 수
num_workers = 4 # 병렬로 실행할 스레드 수
context = 10 # context 창 크기
downsampling = 1e-3 # 빈번한 단어에 대한 다운 샘플링

from gensim.models import word2vec

print("Training model...")
model = word2vec.Word2Vec(sentences,
                         workers =num_workers,
                         size = num_features,
                         min_count = min_word_count,
                         window = context,
                         sample = downsampling)

model.init_sims(replace = True)

model_name = "300features_40minwords_10context"
model.save(model_name)

Training model...
Wall time: 48.1 s


In [119]:
model.most_similar("man")

C:\ProgramData\Anaconda3\envs\gpu_env\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('woman', 0.6614359617233276),
 ('boy', 0.6274638175964355),
 ('doctor', 0.6120471358299255),
 ('soldier', 0.6099045276641846),
 ('person', 0.5613447427749634),
 ('cop', 0.5595858097076416),
 ('scientist', 0.5524453520774841),
 ('priest', 0.5518893003463745),
 ('guy', 0.5511746406555176),
 ('lady', 0.5482226014137268)]

## 문장의 특징벡터 평균을 구해 학습에 활용
- 각 문장에 학습된 단어 등장하면, 각 단어에 해당하는 300차원의 vector를 누적해서 더해주고 마지막으로 모든 단어사전 수로 나눠줌

In [142]:
from gensim.models import Word2Vec
# load model
model = Word2Vec.load("300features_40minwords_10context")


In [150]:
print(type(model.wv.syn0))
# 8315단어 에 대해 300차원의 특징들
print(model.wv.syn0.shape)

<class 'numpy.ndarray'>
(8315, 300)


C:\ProgramData\Anaconda3\envs\gpu_env\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.vectors instead).
  """Entry point for launching an IPython kernel.
C:\ProgramData\Anaconda3\envs\gpu_env\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.vectors instead).
  


In [156]:
len(model.wv.index2word)

8315

In [172]:
model['flower'].shape

C:\ProgramData\Anaconda3\envs\gpu_env\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.


(300,)

In [205]:
import numpy as np

# 각 문장에 해당하는 단어들이 있으면 그 벡터들을 다 더해서 그냥 평균을 내는 특징벡터 생성하는 함수
def makeFeautreVec(words, model, num_features):
    # 단어들의 특징벡터
    featureVec = np.zeros((num_features,), dtype = "float32")
    nwords = 0
    # 단어들
    index2word_set = set(model.wv.index2word)
    # 각 문장에서 단어들 을뽑는데
    for word in words:
        # 그 단어가 단어사전에 있으면
        if word in index2word_set:
            nwords = nwords + 1
            featureVec = np.add(featureVec,model[word])
    featureVec = np.divide(featureVec, nwords)
    return featureVec

# 각 문장에대해 특징벡터평균을 매칭한 행렬 반환
def getAvgFeatureVecs(reviews, model, num_features):
    counter = 0
    reviewFeatureVecs = np.zeros((len(reviews), num_features), dtype = "float32")
    
    for review in reviews:
        if counter%5000 == 0 :
            print("Review {} of {}".format(counter, len(reviews)))
        reviewFeatureVecs[counter] = makeFeautreVec(review, model, num_features)
        
        counter = counter + 1
    return reviewFeatureVecs


In [206]:
%%time
num_features = model.wv.syn0.shape[1]

clean_train_reviews = []
for review in train["review"]:
    clean_train_reviews.append(review_to_wordlist(review, remove_stopwords=False))
trainDataVecs = getAvgFeatureVecs(clean_train_reviews, model, num_features)

clean_test_reviews = []
for review in test["review"]:
    clean_test_reviews.append(review_to_wordlist(review, remove_stopwords=False))
testDataVecs = getAvgFeatureVecs(clean_test_reviews, model, num_features)

C:\ProgramData\Anaconda3\envs\gpu_env\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.vectors instead).
  """Entry point for launching an IPython kernel.


Review 0 of 25000


C:\ProgramData\Anaconda3\envs\gpu_env\lib\site-packages\ipykernel_launcher.py:15: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  from ipykernel import kernelapp as app


Review 5000 of 25000
Review 10000 of 25000
Review 15000 of 25000
Review 20000 of 25000
Review 0 of 25000
Review 5000 of 25000
Review 10000 of 25000
Review 15000 of 25000
Review 20000 of 25000
Wall time: 2min 2s


In [287]:
display(trainDataVecs.shape)

(25000, 300)

In [207]:
from sklearn.ensemble import RandomForestClassifier

forest = RandomForestClassifier(n_estimators=100)
print("Fitting...")
forest = forest.fit(trainDataVecs, train["sentiment"])

result = forest.predict(testDataVecs)

output = pd.DataFrame(data = {"id":test["id"], "sentiment":result})
output.to_csv("../data/movie/Word2Vec_AverageVectors.csv", index = False, quoting=3)

Fitting...


## 문장의 특징벡터 k-means통해 군집화 후 학습에 활용
- 약 8천개정도의 단어리스트들을 1600개정도로 군집화해서 유사한 단어들끼리 뭉쳐줌
- 그리고 

In [215]:
from sklearn.cluster import KMeans

word_vectors = model.wv.syn0
num_clusters = int(word_vectors.shape[0]/5)
print(num_clusters)

1663


C:\ProgramData\Anaconda3\envs\gpu_env\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.vectors instead).
  This is separate from the ipykernel package so we can avoid doing imports until


In [216]:
%%time
# k-means 객체를 초기화 하여 중심점을 추출
kmeans_clustering = KMeans(n_clusters= num_clusters)
idx = kmeans_clustering.fit_predict(word_vectors)

In [218]:
# zip은 동일한 개수로 이루어진 자료형을 묶어줌 
# 각 단어들 군집화해놓은 dictionary 
word_centroid_map = dict(zip(model.wv.index2word, idx))

In [278]:
for cluster in range(0,10):
    print("cluster {}".format(cluster))
    words = []
    for item in word_centroid_map.items(): 
        if item[1] == cluster:
            words.append(item[0])
    print(words)
    

cluster 0
['add', 'create', 'offer', 'deliver', 'provide', 'manage', 'serve', 'contain', 'involve', 'justify', 'lend', 'generate', 'compensate']
cluster 1
['wicked', 'faux', 'gypsy', 'britney', 'pepper']
cluster 2
['air', 'trip', 'road', 'ride', 'step', 'river', 'path', 'remote', 'vacation', 'platform']
cluster 3
['credibility', 'honesty', 'weakness', 'weaknesses', 'strengths', 'characteristics', 'glaring', 'theories', 'traits', 'inaccuracies']
cluster 4
['flynn', 'alec', 'boyle', 'errol', 'dominic', 'omar', 'lara']
cluster 5
['disappointed', 'impressed', 'generous']
cluster 6
['plain', 'painfully', 'downright', 'dialogs', 'intentionally']
cluster 7
['art', 'noir', 'epic', 'parody', 'exploitation', 'mainstream', 'obscure', 'snuff']
cluster 8
['master', 'latest', 'reputation', 'introduction', 'nemesis', 'hanzo']
cluster 9
['native', 'folk', 'greek', 'hippie', 'ethnic', 'angst', 'clash', 'iranian', 'traditions']


In [282]:
# 축소한 단어사전수의 차원을 갖는 행에 누적해서 값을 넣어줌(BOW 느낌)
def create_bag_of_centroids(wordlist, word_centroid_map):
    num_centroids = max(word_centroid_map.values()) + 1
    bag_of_centroids = np.zeros(num_centroids, dtype = "float32")
    
    for word in wordlist:
        if word in word_centroid_map:
            index = word_centroid_map[word]
            bag_of_centroids[index] += 1
    return bag_of_centroids

In [311]:
print(word_centroid_map["and"])

869


In [284]:
# cluster 
train_centroids = np.zeros((train["review"].size, num_clusters), dtype ="float32")
# 각 문장에 대해 (단어사전수/5)크기를 갖는 차원의 특징 vector
counter = 0
for review in clean_train_reviews:
    train_centroids[counter] = create_bag_of_centroids(review, word_centroid_map)
    counter += 1

test_centroids = np.zeros((test["review"].size, num_clusters), dtype = "float32")

counter =0
for review in clean_test_reviews:
    test_centroids[counter] = create_bag_of_centroids(review, word_centroid_map)
    counter += 1


In [286]:
# 기존에는 word2vec을 통해 학습된 모델의 단어 vector들의 shape은 300차원이였는데
# 단어사전수 약 8천개를 5개로 군집화해서 좀더 깊은 차원에 이러한 단어특징 벡터를 
display(train_centroids.shape)

(25000, 1663)

In [291]:
# 랜덤포레스트 학습, 예측
forest = RandomForestClassifier(n_estimators = 100)
print("Fitting a random forest")
forest = forest.fit(train_centroids, train["sentiment"])
result = forest.predict(test_centroids)

output = pd.DataFrame(data = {"id":test["id"], "sentiment":result})
output.to_csv("BagOfCentroids.csv", index = False, quoting=3)

Fitting a random forest
